<a href="https://colab.research.google.com/github/rachel1234561/DS2002_Project/blob/main/converting_and_summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rachel1234561/DS2002_Project.git

Cloning into 'DS2002_Project'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 62 (delta 30), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 4.63 MiB | 3.31 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [2]:
import pandas as pd
import json
import requests
import sqlite3

In [3]:
#the function takes the data being converted and the format of the data
def convert_data(input_data, input_format):
    # make input data a pandas df and handle errors
    if input_format == 'csv':
        #save csv as df
        df = pd.read_csv(input_data)
        #print a summary of the data file ingestion
        num_records = len(df)
        num_columns = len(df.columns)
        print("Summary of the data file ingestion:")
        print("Number of records:" + str(num_records))
        print("Number of columns:" + str(num_columns))
        print("")
    elif input_format == 'json':
        #save json as df
        data = json.loads(input_data)
        #code below resolves the issue of data dictionaries within data dictionaries
        columns_data = data['meta']['view']['columns']
        columns = [column['name'] for column in columns_data]
        df = pd.DataFrame(data['data'], columns=columns)
        #print a summary of the data file ingestion
        print("summary of the data file ingestion:\n")
        print("Number of records:" + str(len(df)))
        print("Number of columns:" + str(len(df.columns)))
    else:
        print("Invalid input format. Choose 'csv' or 'json'.")

    # column deletion
    print(f"Columns: {df.columns.tolist()}\n")
    # columns to keep/drop
    drop_columns = input("Enter comma-separated column names to drop, or leave blank to keep all: ")
    if drop_columns:
        drop_columns = [col.strip() for col in drop_columns.split(',')]
        df = df.drop(columns=drop_columns, errors='ignore')
        print(f"Columns after dropping: {df.columns.tolist()}")

    #ask use to input their prefered file type
    output_format = input('Enter output format (csv, json, sql): ')
    # Output data as the correct file type
    #saves a csv to files
    if output_format == 'csv':
        df.to_csv('output.csv')
        # Post-processing summary
        print("Post-processing summary:")
        print("Number of records:", len(pd.read_csv('output.csv')))
        print("Number of columns:", len(pd.read_csv('output.csv').columns))
    #saves a json file to files (also might need to debug this part)
    elif output_format == 'json':
        df.to_json('output.json')
        # Post-processing summary (I used stack overflow to debug this part)
        with open('output.json', 'r') as f:
         data = json.load(f)
        print("Post-processing summary:")
        print("Number of records:", len(df))
        print("Number of columns:", len(data))
    #saves to a sql db + db file in files
    elif output_format == 'sql':
      connection = sqlite3.connect('output.db')
      cursor = connection.cursor()
      connection.close()
      # Post-processing summary (SQL)
      connection = sqlite3.connect('output.db')
      cursor = connection.cursor()
      # ensure data type is compatable with sql
      for column in df.columns:
        if df[column].dtype not in [int, float, bool, object,str]:
            df[column] = df[column].astype(str)
        df[column] = df[column].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x) #I used ai debug to add this line
      df.to_sql('output', connection, if_exists='replace', index=False)
      cursor.execute("SELECT COUNT(*) FROM output")
      num_records = cursor.fetchone()[0]
      cursor.execute("SELECT COUNT(*) FROM pragma_table_info('output')")
      num_columns = cursor.fetchone()[0]
      connection.close()
      print("Post-processing summary (SQL):")
      print("Number of records:", num_records)
      print("Number of columns:", num_columns)
    else:
        print("Invalid output format. Choose 'csv', 'json', or 'sql'.")

In [4]:
#pass the path to the csv to the function
convert_data('/content/DS2002_Project/cereal.csv', 'csv')

Summary of the data file ingestion:
Number of records:77
Number of columns:16

Columns: ['name', 'mfr', 'type', 'calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'shelf', 'weight', 'cups', 'rating']

Enter comma-separated column names to drop, or leave blank to keep all: fiber
Columns after dropping: ['name', 'mfr', 'type', 'calories', 'protein', 'fat', 'sodium', 'carbo', 'sugars', 'potass', 'vitamins', 'shelf', 'weight', 'cups', 'rating']
Enter output format (csv, json, sql): json
Post-processing summary:
Number of records: 77
Number of columns: 15


In [7]:
#pass json to the file
#this is an api from data.gov that has adoptoble pets from Montgomery County of Maryland?
url = 'https://data.montgomerycountymd.gov/api/views/e54u-qx42/rows.json?accessType=DOWNLOAD'
response = requests.get(url)
convert_data(response.text, 'json')

summary of the data file ingestion:

Number of records:67
Number of columns:25
Columns: ['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta', 'Animal ID', 'Intake Type', 'In Date', 'Pet name', 'Animal Type', 'Pet Age', 'Pet Size', 'Color', 'Breed', 'Sex', 'URL Link ', 'Crossing', 'Council Districts', 'Councils', 'Communities', 'Zip Codes', 'Municipalities']

Enter comma-separated column names to drop, or leave blank to keep all: created_at
Columns after dropping: ['sid', 'id', 'position', 'created_meta', 'updated_at', 'updated_meta', 'meta', 'Animal ID', 'Intake Type', 'In Date', 'Pet name', 'Animal Type', 'Pet Age', 'Pet Size', 'Color', 'Breed', 'Sex', 'URL Link ', 'Crossing', 'Council Districts', 'Councils', 'Communities', 'Zip Codes', 'Municipalities']
Enter output format (csv, json, sql): sql
Post-processing summary (SQL):
Number of records: 67
Number of columns: 24


In [8]:
#this is to test to see if the SQL is working
connection = sqlite3.connect('output.db')
cursor = connection.cursor()
cursor.execute("SELECT * FROM output")
rows = cursor.fetchall()
for row in rows:
    print(row)
connection.close()

('row-ggrz-t92v.hmx5', '00000000-0000-0000-9381-CD0B9342B326', 0, None, 1726437633, None, '{ }', 'A523470', 'OWNER SUR', '2024-09-15T00:00:00', 'NORMAN', 'OTHER', '3 YEARS', 'MED', 'TRICOLOR', 'GUINEA PIG', 'M', '["http://www.petharbor.com/get_image.asp?res=DETAIL&id=A523470&location=MONT", null]', None, None, None, None, None, None)
('row-ydq3.5gqi-jqax', '00000000-0000-0000-EB42-8398A52B1875', 0, None, 1726596036, None, '{ }', 'A523467', 'OWNER SUR', '2024-09-15T00:00:00', 'RODNEY', 'OTHER', '3 YEARS', 'MED', 'BROWN / WHITE', 'GUINEA PIG', 'M', '["http://www.petharbor.com/get_image.asp?res=DETAIL&id=A523467&location=MONT", null]', None, None, None, None, None, None)
('row-34rj~3y9n~4n8k', '00000000-0000-0000-9E8F-12A9B9B03884', 0, None, 1727539234, None, '{ }', 'A425626', 'OWNER SUR', '2024-09-17T00:00:00', 'TOBY', 'DOG', '8 YEARS', 'LARGE', 'TAN / WHITE', 'AM PIT BULL TER / MIX', 'N', '["http://www.petharbor.com/get_image.asp?res=DETAIL&id=A425626&location=MONT", null]', None, None,

In [ ]:
#this is to test to see if the json part is working
#I used stack to figure out how to show the output.json in pretty print
with open('output.json', 'r') as f:
  data = json.load(f)
print(json.dumps(data, indent=1))